# Import Packages 

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Scraped Postal Codes 

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0] # scrape table
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [3]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop = True) # only process cells with assigned borough
df.dtypes

Postal Code      object
Borough          object
Neighbourhood    object
dtype: object

In [4]:
df['Neighbourhood'].value_counts()

Downsview                                          4
Don Mills                                          2
Kennedy Park, Ionview, East Birchmount Park        1
Lawrence Manor, Lawrence Heights                   1
Guildwood, Morningside, West Hill                  1
                                                  ..
Northwest, West Humber - Clairville                1
Christie                                           1
Stn A PO Boxes                                     1
Forest Hill North & West, Forest Hill Road Park    1
Parkwoods                                          1
Name: Neighbourhood, Length: 99, dtype: int64

# Filtered Postal Codes

In [5]:
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [115]:
df.head(20)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Get Postalcode Latitude, Longitudes 

In [6]:
postal_latlon = pd.read_csv('https://cocl.us/Geospatial_data')
postal_latlon.info()
postal_latlon.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Postal Code  103 non-null    object 
 1   Latitude     103 non-null    float64
 2   Longitude    103 non-null    float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Join the two tables 

In [210]:
toronto = df.merge(postal_latlon, how='left', on='Postal Code')
toronto.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


###  Find neighborhood centers

# Explore the Data

### Credentials 

In [243]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [242]:
neighborhood_latitude = toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


mean neighborhood size of 0.32 square miles \
mile = 1609 m \
size = 828796.2 square meters
r = (s2/ π)1/2

In [240]:
radius = (828796.2 / np.pi)**0.5
radius

513.6282937023215

### Queue a single loc's venues

In [14]:
limit = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    limit)

'https://api.foursquare.com/v2/venues/explore?&client_id=V5WNA3NTS1EDMKBFBD3XMF5I5V4CRX0B2TJCCL3HRK4WCML2&client_secret=EZGTAB3W5IHD2YACIXXCQPOFY1RLSDSMQ2SLRVEDAIYZIPFG&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [250]:
with open(r'E:\ClassesOnline\Coursera\ibm\course9_appliedcapstone\credentials.json') as file:
    data = json.load(file)
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        data['CLIENT_ID'], 
        data['CLIENT_SECRET'], 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius, 
        limit)
    results = requests.get(url).json()
    
    url = None # remove credentials
    data = None
results

{'meta': {'code': 200, 'requestId': '5fb0561a92e219640d83b542'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757875604617006,
    'lng': -79.32327656508535},
   'sw': {'lat': 43.748641595383, 'lng': -79.33603643491465}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs'

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.751976,-79.332140
1,Variety Store,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",43.751974,-79.333114


## Get all venues

In [251]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('[',name,']', end=', ')
            
        # protect credentials
        with open(r'E:\ClassesOnline\Coursera\ibm\course9_appliedcapstone\credentials.json') as file:
            data = json.load(file)
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                data['CLIENT_ID'], 
                data['CLIENT_SECRET'], 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
            
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # split neighborhoods back up
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [252]:
# type your answer here
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

[ Parkwoods ], [ Victoria Village ], [ Regent Park, Harbourfront ], [ Lawrence Manor, Lawrence Heights ], [ Queen's Park, Ontario Provincial Government ], [ Islington Avenue, Humber Valley Village ], [ Malvern, Rouge ], [ Don Mills ], [ Parkview Hill, Woodbine Gardens ], [ Garden District, Ryerson ], [ Glencairn ], [ West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale ], [ Rouge Hill, Port Union, Highland Creek ], [ Don Mills ], [ Woodbine Heights ], [ St. James Town ], [ Humewood-Cedarvale ], [ Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood ], [ Guildwood, Morningside, West Hill ], [ The Beaches ], [ Berczy Park ], [ Caledonia-Fairbanks ], [ Woburn ], [ Leaside ], [ Central Bay Street ], [ Christie ], [ Cedarbrae ], [ Hillcrest Village ], [ Bathurst Manor, Wilson Heights, Downsview North ], [ Thorncliffe Park ], [ Richmond, Adelaide, King ], [ Dufferin, Dovercourt Village ], [ Scarborough Village ], [ Fairview, Henry Farm, Oriole ], [ Northwood Park, 

In [28]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
...,...,...,...,...,...,...,...
2134,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2135,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Once Upon A Child,43.631075,-79.518290,Kids Store
2136,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Value Village,43.631269,-79.518238,Thrift / Vintage Store
2137,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


In [35]:
toronto_venues.groupby('Neighborhood').size().reset_index(name='counts')

,Neighborhood,counts
0,Agincourt,5
1,"Alderwood, Long Branch",7
2,"Bathurst Manor, Wilson Heights, Downsview North",21
3,Bayview Village,4
4,"Bedford Park, Lawrence Manor East",22
...,...,...
91,"Willowdale, Willowdale West",5
92,Woburn,4
93,Woodbine Heights,8
94,York Mills West,2


In [162]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


In [191]:
toronto_venues = toronto_venues[toronto_venues['Venue Category'] != 'Neighborhood']
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
...,...,...,...,...,...,...,...
2134,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2135,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Once Upon A Child,43.631075,-79.518290,Kids Store
2136,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Value Village,43.631269,-79.518238,Thrift / Vintage Store
2137,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


In [189]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.shape

(2135, 273)

In [190]:
# add neighborhood column back to dataframe
toronto_onehot.insert(loc = 0, column = 'Neighborhood', value = toronto_venues['Neighborhood'])
# move neighborhood column to the first column

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [192]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [194]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1             Breakfast Spot   0.2
2                     Lounge   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


----Alderwood, Long Branch----
         venue  freq
0  Pizza Place  0.29
1          Gym  0.14
2  Coffee Shop  0.14
3          Pub  0.14
4     Pharmacy  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0               Bank  0.10
1        Coffee Shop  0.10
2           Pharmacy  0.05
3  Mobile Phone Shop  0.05
4        Gas Station  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1         Coffee Shop  0.09
2  Italian Restaurant  0.09
3            Pharmacy  0.05
4       G

4  Molecular Gastronomy Restaurant   0.0


----Humewood-Cedarvale----
          venue  freq
0       Dog Run   0.2
1         Trail   0.2
2  Hockey Arena   0.2
3         Field   0.2
4  Tennis Court   0.2


----India Bazaar, The Beaches West----
                venue  freq
0                Park  0.11
1         Pizza Place  0.05
2  Italian Restaurant  0.05
3             Brewery  0.05
4   Fish & Chips Shop  0.05


----Kennedy Park, Ionview, East Birchmount Park----
                 venue  freq
0           Hobby Shop  0.25
1        Train Station  0.25
2     Department Store  0.25
3          Coffee Shop  0.25
4  Moroccan Restaurant  0.00


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                    Coffee Shop  0.05
1  Vegetarian / Vegan Restaurant  0.05
2                            Bar  0.05
3                           Café  0.05
4             Mexican Restaurant  0.05


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Garde

                   venue  freq
0            Pizza Place   0.2
1  Portuguese Restaurant   0.2
2           Intersection   0.2
3            Coffee Shop   0.2
4           Hockey Arena   0.2


----West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale----
                             venue  freq
0                       Print Shop   1.0
1                Accessories Store   0.0
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Westmount----
                venue  freq
0         Pizza Place  0.17
1        Intersection  0.17
2      Sandwich Place  0.17
3  Chinese Restaurant  0.17
4         Coffee Shop  0.17


----Weston----
                             venue  freq
0                             Park   1.0
1                Accessories Store   0.0
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Wexford, Maryvale----
        

## Fit K cluster model 

In [233]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([8, 0, 8, 8, 0, 8, 8, 8, 8, 8])

In [234]:
tlookup = pd.concat([toronto_grouped['Neighborhood'], pd.Series(kmeans.labels_, name = 'Category')], axis = 1)
tlookup

,Neighborhood,Category
0,Agincourt,8
1,"Alderwood, Long Branch",0
2,"Bathurst Manor, Wilson Heights, Downsview North",8
3,Bayview Village,8
4,"Bedford Park, Lawrence Manor East",0
...,...,...
91,"Willowdale, Willowdale West",0
92,Woburn,8
93,Woodbine Heights,8
94,York Mills West,6


In [235]:
# 1 neighborhood -> >1 postal code
toronto_merged = tlookup.merge(toronto, how = 'left', on = 'Neighborhood')
toronto_merged

,Neighborhood,Category,Postal Code,Borough,Latitude,Longitude
0,Agincourt,8,M1S,Scarborough,43.794200,-79.262029
1,"Alderwood, Long Branch",0,M8W,Etobicoke,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",8,M3H,North York,43.754328,-79.442259
3,Bayview Village,8,M2K,North York,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",0,M5M,North York,43.733283,-79.419750
...,...,...,...,...,...,...
95,"Willowdale, Willowdale West",0,M2R,North York,43.782736,-79.442259
96,Woburn,8,M1G,Scarborough,43.770992,-79.216917
97,Woodbine Heights,8,M4C,East York,43.695344,-79.318389
98,York Mills West,6,M2P,North York,43.752758,-79.400049


# Create map of clusters 

In [236]:
# create map
latitude = toronto_merged['Latitude'].mean()
longitude = toronto_merged['Longitude'].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Category']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Additional Steps:
1. Run again on a larger radius
    - Remove venues that were in the smaller radius -> Outer ring of venues
2. Use distances between neighboorhoods (ex. how close are the X nearest neighborhoods?) for radius when queuing FourSquare

In [237]:
# for sharing
CLIENT_ID = None
CLIENT_SECRET = None